# 🔄 완전한 재시작 가능한 Grid Search
## **진짜 작동하는 파이썬 코드 - 중단된 지점부터 완벽 재시작!**

### 🎯 **실제 기능**
- ✅ **실시간 저장**: 100개 조합마다 자동 체크포인트 저장
- ✅ **완벽한 재시작**: 정확한 중단 지점부터 재개
- ✅ **실시간 모니터링**: 진행률, 최고 성능, 예상 완료 시간
- ✅ **안전한 중단**: Ctrl+C로 언제든 안전하게 중단

### ⚡ **sklearn vs 재시작 Grid Search**
| 상황 | sklearn GridSearchCV | 재시작 가능한 Grid Search |
|------|---------------------|---------------------------|
| 6시간 진행 후 중단 | ❌ 6시간 완전 손실 | ✅ 최대 50분만 재계산 |
| 재시작 | ❌ 처음부터 다시 | ✅ 즉시 중단 지점부터 |
| 중간 결과 | ❌ 접근 불가 | ✅ 실시간 확인 |

In [ ]:
# 📦 라이브러리 임포트 및 데이터 로드
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
import time
import os
import json
import itertools
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🔄 재시작 가능한 Grid Search 시스템 초기화")
print("=" * 60)

# 데이터 로드
print("📊 데이터 로드 중...")
with open('../../data/processed/df_selected_05.pkl', 'rb') as f:
    df = pickle.load(f)

y = df['SalePrice']
X = df.drop('SalePrice', axis=1)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

print(f"   데이터 형태: X {X.shape}, y {y.shape}")

# 파라미터 그리드 정의 (9,600개 조합)
param_grid = {
    'n_estimators': [500, 600, 650, 700, 750, 800],           # 6개
    'max_depth': [None, 10, 25, 35],                          # 4개 
    'min_samples_split': [3, 5, 7, 10, 13],                  # 5개
    'min_samples_leaf': [2, 3, 4, 5, 6],                     # 5개
    'max_features': [0.3, 0.5, 0.7, 'sqrt'],                 # 4개
    'max_samples': [0.65, 0.7, 0.8, 0.9],                    # 4개
    'bootstrap': [True]                                        # 1개 (고정)
}

# 모든 조합 생성
param_names = list(param_grid.keys())
param_values = list(param_grid.values())ㅣ
all_combinations = list(itertools.product(*param_values))
total_combinations = len(all_combinations)

print(f"🔢 총 조합 수: {total_combinations:,}개")

# 기준점 설정
baseline_r2 = 0.8335
random_search_best = 0.8426

print(f"📈 성능 기준점:")
print(f"   기준선 R²: {baseline_r2:.4f}")
print(f"   Random Search 최고: {random_search_best:.4f}")
print(f"\n✅ 초기화 완료!")
print("=" * 60)

🔄 재시작 가능한 Grid Search 시스템 초기화
📊 데이터 로드 중...
   데이터 형태: X (1460, 15), y (1460,)
🔢 총 조합 수: 9,600개
📈 성능 기준점:
   기준선 R²: 0.8335
   Random Search 최고: 0.8426

✅ 초기화 완료!


In [10]:
# 🔄 재시작 가능한 Grid Search 클래스 (핵심 구현!)

class ResumableGridSearch:
    """중단된 지점부터 재시작 가능한 Grid Search"""
    
    def __init__(self, param_combinations, param_names, X, y, cv, scoring='r2', 
                 checkpoint_file='resumable_checkpoint.json',
                 results_file='resumable_results.pkl'):
        
        self.param_combinations = param_combinations
        self.param_names = param_names
        self.X = X
        self.y = y
        self.cv = cv
        self.scoring = scoring
        self.checkpoint_file = checkpoint_file
        self.results_file = results_file
        
        self.total_combinations = len(param_combinations)
        self.completed_combinations = 0
        self.best_score = -np.inf
        self.best_params = None
        self.all_results = []
        self.start_time = None
        
        # 기존 체크포인트 로드 시도
        self.load_checkpoint()
    
    def load_checkpoint(self):
        """기존 체크포인트에서 진행 상황 복원"""
        if os.path.exists(self.checkpoint_file):
            try:
                with open(self.checkpoint_file, 'r') as f:
                    checkpoint = json.load(f)
                
                self.completed_combinations = checkpoint.get('completed_combinations', 0)
                self.best_score = checkpoint.get('best_score', -np.inf)
                self.best_params = checkpoint.get('best_params', None)
                self.all_results = checkpoint.get('all_results', [])
                
                print(f"🔄 체크포인트 로드 성공!")
                print(f"   완료된 조합: {self.completed_combinations:,}개")
                print(f"   진행률: {self.completed_combinations/self.total_combinations*100:.1f}%")
                print(f"   현재 최고 점수: {self.best_score:.6f}")
                
                if self.completed_combinations > 0:
                    print(f"   ✅ {self.completed_combinations+1}번째 조합부터 재시작 가능!")
                
            except Exception as e:
                print(f"⚠️ 체크포인트 로드 실패: {str(e)}")
                print(f"   → 처음부터 새로 시작합니다.")
                self._reset_state()
        else:
            print(f"📝 새로운 Grid Search 시작")
            self._reset_state()
    
    def _reset_state(self):
        """상태 초기화"""
        self.completed_combinations = 0
        self.best_score = -np.inf
        self.best_params = None
        self.all_results = []
    
    def save_checkpoint(self):
        """체크포인트 저장"""
        checkpoint_data = {
            'timestamp': datetime.now().isoformat(),
            'completed_combinations': self.completed_combinations,
            'total_combinations': self.total_combinations,
            'progress_percentage': self.completed_combinations / self.total_combinations * 100,
            'best_score': float(self.best_score),
            'best_params': self.best_params,
            'all_results': self.all_results,
            'elapsed_time_seconds': (datetime.now() - self.start_time).total_seconds() if self.start_time else 0
        }
        
        try:
            with open(self.checkpoint_file, 'w') as f:
                json.dump(checkpoint_data, f, indent=2, default=str)
            return True
        except Exception as e:
            print(f"⚠️ 체크포인트 저장 실패: {str(e)}")
            return False
    
    def save_results(self):
        """결과 파일 저장"""
        results_data = {
            'best_score': self.best_score,
            'best_params': self.best_params,
            'all_results': pd.DataFrame(self.all_results) if self.all_results else pd.DataFrame(),
            'total_combinations': self.total_combinations,
            'completed_combinations': self.completed_combinations,
            'completion_time': datetime.now().isoformat(),
            'total_time_seconds': (datetime.now() - self.start_time).total_seconds() if self.start_time else 0
        }
        
        try:
            with open(self.results_file, 'wb') as f:
                pickle.dump(results_data, f)
            return True
        except Exception as e:
            print(f"⚠️ 결과 저장 실패: {str(e)}")
            return False

print("✅ ResumableGridSearch 클래스 정의 완료!")

✅ ResumableGridSearch 클래스 정의 완료!


In [12]:
# 🚀 Grid Search 실행 메서드 (핵심 기능!)

def fit(self, verbose=True, save_interval=100):
    """재시작 가능한 Grid Search 실행"""
    
    if self.start_time is None:
        self.start_time = datetime.now()
    
    print(f"\n🚀 재시작 가능한 Grid Search 시작!")
    print(f"   시작 지점: {self.completed_combinations+1:,}번째 조합")
    print(f"   남은 조합: {self.total_combinations - self.completed_combinations:,}개")
    print(f"   체크포인트 저장: {save_interval}개 조합마다")
    print(f"=" * 50)
    
    try:
        # 중단된 지점부터 시작!
        for i in range(self.completed_combinations, self.total_combinations):
            combination = self.param_combinations[i]
            
            # 파라미터 딕셔너리 생성
            params = dict(zip(self.param_names, combination))
            
            # Random Forest 모델 생성 및 평가
            rf = RandomForestRegressor(random_state=42, n_jobs=-1, **params)
            
            combination_start = time.time()
            scores = cross_val_score(rf, self.X, self.y, cv=self.cv, scoring=self.scoring)
            combination_end = time.time()
            
            mean_score = scores.mean()
            std_score = scores.std()
            
            # 결과 저장
            result = {
                'combination_index': i,
                'params': params,
                'mean_test_score': float(mean_score),
                'std_test_score': float(std_score),
                'fit_time': combination_end - combination_start,
                'timestamp': datetime.now().isoformat()
            }
            
            self.all_results.append(result)
            self.completed_combinations = i + 1
            
            # 최고 점수 업데이트
            if mean_score > self.best_score:
                self.best_score = mean_score
                self.best_params = params.copy()
                if verbose:
                    print(f"🎯 새로운 최고 점수: {mean_score:.6f} (조합 {i+1:,})")
                    print(f"   파라미터: {params}")
            
            # 진행 상황 출력
            if verbose and (i + 1) % save_interval == 0:
                progress = (i + 1) / self.total_combinations * 100
                elapsed = (datetime.now() - self.start_time).total_seconds()
                
                # 현재 세션에서 완료된 조합 수로 예상 시간 계산
                session_completed = i + 1 - (self.completed_combinations - len(self.all_results))
                if session_completed > 0:
                    avg_time = elapsed / session_completed
                    remaining = self.total_combinations - i - 1
                    est_time = avg_time * remaining
                    
                    print(f"📊 진행 상황: {i+1:,}/{self.total_combinations:,} ({progress:.1f}%)")
                    print(f"   현재 최고 점수: {self.best_score:.6f}")
                    print(f"   경과 시간: {elapsed/3600:.1f}시간")
                    print(f"   예상 남은 시간: {est_time/3600:.1f}시간")
                    print(f"   예상 완료: {(datetime.now() + timedelta(seconds=est_time)).strftime('%m-%d %H:%M')}")
                    print(f"-" * 40)
            
            # 정기적으로 체크포인트 저장 (중요!)
            if (i + 1) % save_interval == 0:
                if self.save_checkpoint():
                    if verbose and (i + 1) % (save_interval * 2) == 0:
                        print(f"💾 체크포인트 저장 완료 (조합 {i+1:,})")
                
            # 100개 조합마다 결과 파일도 저장
            if (i + 1) % 100 == 0:
                self.save_results()
        
        # 최종 저장
        self.save_checkpoint()
        self.save_results()
        
        print(f"\n🎉 Grid Search 완료!")
        print(f"   최고 R² 점수: {self.best_score:.6f}")
        print(f"   Random Search 대비: {self.best_score - random_search_best:+.6f}")
        print(f"   기준선 대비: {self.best_score - baseline_r2:+.6f}")
        print(f"   총 소요 시간: {(datetime.now() - self.start_time).total_seconds()/3600:.1f}시간")
        
        print(f"\n🏆 최적 하이퍼파라미터:")
        for param, value in self.best_params.items():
            print(f"   {param}: {value}")
            
    except KeyboardInterrupt:
        print(f"\n🛑 사용자에 의해 중단되었습니다!")
        print(f"   완료된 조합: {self.completed_combinations:,}개")
        print(f"   진행률: {self.completed_combinations/self.total_combinations*100:.1f}%")
        print(f"   현재 최고 점수: {self.best_score:.6f}")
        
        # 중단 시에도 저장
        if self.save_checkpoint():
            print(f"   💾 체크포인트 저장 완료!")
        self.save_results()
        
        print(f"\n🔄 재시작 방법:")
        print(f"   동일한 코드를 다시 실행하면")
        print(f"   자동으로 {self.completed_combinations+1:,}번째 조합부터 재개됩니다!")
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {str(e)}")
        
        # 오류 시에도 저장
        self.save_checkpoint()
        self.save_results()
        
        print(f"🔄 문제 해결 후 재시작하면 {self.completed_combinations+1:,}번째부터 재개됩니다.")

# 클래스에 fit 메서드 추가
ResumableGridSearch.fit = fit

print("✅ Grid Search 실행 메서드 추가 완료!")

✅ Grid Search 실행 메서드 추가 완료!


In [20]:
# 🚀 실제 실행 코드 - 여기서 True/False로 제어!

# 실행 제어 변수 - True로 변경하면 실행됩니다!
EXECUTE_RESUMABLE_SEARCH = True

# 체크포인트 디렉토리 생성
checkpoint_dir = 'checkpoint_files'
os.makedirs(checkpoint_dir, exist_ok=True)

# 파일명 설정 (기존 파일 재사용 시 여기서 변경하세요!)
CHECKPOINT_FILE = f'{checkpoint_dir}/resumable_checkpoint_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
RESULTS_FILE = f'{checkpoint_dir}/resumable_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pkl'

print("🎯 재시작 가능한 Grid Search 실행 준비")
print("=" * 50)

if EXECUTE_RESUMABLE_SEARCH:
    print("🚀 9,600개 조합 Grid Search 실행 중...")
    
    # 재시작 가능한 Grid Search 인스턴스 생성
    grid_search = ResumableGridSearch(
        param_combinations=all_combinations,
        param_names=param_names,
        X=X,
        y=y,
        cv=kfold,
        scoring='r2',
        checkpoint_file=CHECKPOINT_FILE,
        results_file=RESULTS_FILE
    )
    
    # Grid Search 실행 - Ctrl+C로 중단하면 자동 저장!
    grid_search.fit(
        verbose=True,         # 진행 상황 실시간 출력
        save_interval=100     # 100개 조합마다 체크포인트 저장
    )
    
else:
    print("🛑 Grid Search 실행이 비활성화되어 있습니다.")
    print("\n▶️ 실행하려면:")
    print("   EXECUTE_RESUMABLE_SEARCH = True")
    print("   로 변경 후 이 셀을 다시 실행하세요.")
    
    print(f"\n📁 현재 파일 설정:")
    print(f"   체크포인트: {CHECKPOINT_FILE}")
    print(f"   결과 파일: {RESULTS_FILE}")
    
    print(f"\n💡 재시작 가능한 Grid Search 장점:")
    print(f"   ✅ 100개 조합마다 자동 저장 → 최대 100개만 손실")
    print(f"   ✅ Ctrl+C로 안전하게 중단 가능") 
    print(f"   ✅ 재실행 시 중단 지점부터 자동 재개")
    print(f"   ✅ 실시간 진행률 & 최고 성능 추적")
    print(f"   ✅ 예상 완료 시간 실시간 계산")
    print(f"   📁 체크포인트 파일: {checkpoint_dir}/ 디렉토리")
    
    # 기존 체크포인트 파일 확인
    if os.path.exists(checkpoint_dir):
        checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('resumable_checkpoint_')]
    else:
        checkpoint_files = []
    if checkpoint_files:
        print(f"\n📁 기존 체크포인트 파일 발견: {len(checkpoint_files)}개")
        for file in sorted(checkpoint_files, reverse=True)[:3]:  # 최신 3개만 표시
            try:
                file_path = os.path.join(checkpoint_dir, file)
                with open(file_path, 'r') as f:
                    checkpoint = json.load(f)
                completed = checkpoint.get('completed_combinations', 0)
                total = checkpoint.get('total_combinations', total_combinations)
                progress = completed / total * 100
                best_score = checkpoint.get('best_score', 0)
                timestamp = checkpoint.get('timestamp', '알 수 없음')[:16]  # 날짜만
                
                print(f"\n   📄 {file}")
                print(f"      날짜: {timestamp}")
                print(f"      진행률: {completed:,}/{total:,} ({progress:.1f}%)")
                print(f"      최고 점수: {best_score:.6f}")
                print(f"      ✅ 이 파일로 재시작 가능!")
            except:
                print(f"   ❓ {file} (읽기 실패)")
                
        print(f"\n🔄 기존 체크포인트로 재시작하려면:")
        print(f"   1. 위에서 CHECKPOINT_FILE = '{checkpoint_dir}/원하는_파일명.json' 변경")
        print(f"   2. EXECUTE_RESUMABLE_SEARCH = True 설정")
        print(f"   3. 셀 재실행 → 자동으로 중단 지점부터 재개!")
    else:
        print(f"\n📋 기존 체크포인트 파일 없음")
        print(f"   → 새로 시작하면 1번째 조합부터 실행됩니다.")

print(f"\n" + "="*50)


🎯 재시작 가능한 Grid Search 실행 준비
🚀 9,600개 조합 Grid Search 실행 중...
📝 새로운 Grid Search 시작

🚀 재시작 가능한 Grid Search 시작!
   시작 지점: 1번째 조합
   남은 조합: 9,600개
   체크포인트 저장: 100개 조합마다
🎯 새로운 최고 점수: 0.842999 (조합 1)
   파라미터: {'n_estimators': 500, 'max_depth': None, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 0.3, 'max_samples': 0.65, 'bootstrap': True}
🎯 새로운 최고 점수: 0.843078 (조합 42)
   파라미터: {'n_estimators': 500, 'max_depth': None, 'min_samples_split': 3, 'min_samples_leaf': 4, 'max_features': 0.7, 'max_samples': 0.7, 'bootstrap': True}
📊 진행 상황: 100/9,600 (1.0%)
   현재 최고 점수: 0.843078
   경과 시간: 0.1시간
   예상 남은 시간: 7.7시간
   예상 완료: 07-23 08:17
----------------------------------------
📊 진행 상황: 200/9,600 (2.1%)
   현재 최고 점수: 0.843078
   경과 시간: 0.2시간
   예상 남은 시간: 7.4시간
   예상 완료: 07-23 08:00
----------------------------------------
💾 체크포인트 저장 완료 (조합 200)
🎯 새로운 최고 점수: 0.843135 (조합 242)
   파라미터: {'n_estimators': 500, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 0

In [16]:
# 🧪 빠른 테스트 코드 (소규모로 먼저 테스트!)

# 체크포인트 디렉토리 설정 (테스트용)
checkpoint_dir = 'checkpoint_files'
os.makedirs(checkpoint_dir, exist_ok=True)

# 테스트 실행 여부 - 작은 규모로 먼저 테스트해보세요!
RUN_TEST = True

if RUN_TEST:
    print("🧪 소규모 테스트 Grid Search 실행 중...")
    
    # 테스트용 작은 파라미터 그리드 (12개 조합)
    test_param_grid = {
        'n_estimators': [500, 600],              # 2개
        'max_depth': [None, 25],                 # 2개  
        'min_samples_split': [5, 10],            # 2개
        'min_samples_leaf': [3],                 # 1개 (고정)
        'max_features': [0.7],                   # 1개 (고정)
        'max_samples': [0.8],                    # 1개 (고정)
        'bootstrap': [True]                      # 1개 (고정)
    }
    
    # 테스트용 조합 생성
    test_param_names = list(test_param_grid.keys())
    test_param_values = list(test_param_grid.values())
    test_combinations = list(itertools.product(*test_param_values))
    
    print(f"📊 테스트 조합 수: {len(test_combinations)}개 (약 2-3분 소요)")
    
    # 테스트 Grid Search 실행
    test_search = ResumableGridSearch(
        param_combinations=test_combinations,
        param_names=test_param_names,
        X=X,
        y=y,
        cv=kfold,
        scoring='r2',
        checkpoint_file=f'{checkpoint_dir}/test_checkpoint.json',
        results_file=f'{checkpoint_dir}/test_results.pkl'
    )
    
    # 테스트 실행 (5개 조합마다 저장)
    print("\\n🧪 테스트 시작 - 중간에 Ctrl+C로 중단해보세요!")
    test_search.fit(verbose=True, save_interval=5)
    
    print("\\n✅ 테스트 완료! 이제 본격적인 9,600개 조합을 실행할 수 있습니다.")
    
else:
    print("🧪 테스트 모드 비활성화")
    print("\\n💡 처음 사용하시나요?")
    print("   RUN_TEST = True로 변경하여 먼저 12개 조합으로 테스트해보세요!")
    print("   테스트에서는 재시작 기능을 안전하게 확인할 수 있습니다.")
    print("   (약 2-3분 소요, 중간에 Ctrl+C로 중단 테스트 가능)")

print("\\n" + "="*50)


🧪 소규모 테스트 Grid Search 실행 중...
📊 테스트 조합 수: 8개 (약 2-3분 소요)
🔄 체크포인트 로드 성공!
   완료된 조합: 8개
   진행률: 100.0%
   현재 최고 점수: 0.843093
   ✅ 9번째 조합부터 재시작 가능!
\n🧪 테스트 시작 - 중간에 Ctrl+C로 중단해보세요!

🚀 재시작 가능한 Grid Search 시작!
   시작 지점: 9번째 조합
   남은 조합: 0개
   체크포인트 저장: 5개 조합마다

🎉 Grid Search 완료!
   최고 R² 점수: 0.843093
   Random Search 대비: +0.000493
   기준선 대비: +0.009593
   총 소요 시간: 0.0시간

🏆 최적 하이퍼파라미터:
   n_estimators: 500
   max_depth: None
   min_samples_split: 10
   min_samples_leaf: 3
   max_features: 0.7
   max_samples: 0.8
   bootstrap: True
\n✅ 테스트 완료! 이제 본격적인 9,600개 조합을 실행할 수 있습니다.
\n==================================================
